In [15]:
### import libraries
import gensim
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import gensim.downloader as api

In [3]:
## api load word2vec mode

#wv = api.load('glove-wiki-gigaword-100')

In [4]:
## dataset
import pandas as pd
messages = pd.read_csv('data/SMSSpamCollection.csv',sep='\t',names=['label','message'])

messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
## lemmatizer libbrary
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [6]:
## preprocessing

corpus = []

for i in range(0, len(messages)):
    review = re.sub('{^a-zA-Z}',' ',messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [50]:
#corpus

In [52]:
## simple preprpcess to separate words in sentence

from nltk import sent_tokenize
from gensim.utils import simple_preprocess

words = []
for sent in corpus:
    #sent_tokens = sent_tokenize(sent)
    #for sent in sent_tokens:
    words.append(simple_preprocess(sent))

print(f"\nTotal messages: {len(words)}")
print(f"Sample tokenized message: {words[0]}")


Total messages: 5572
Sample tokenized message: ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']


In [33]:
## train word2vec from scratch

model = gensim.models.Word2Vec(words)

In [53]:
## to get all the vocabulary

print(f"Vocabulary size: {len(model.wv.index_to_key)}")
print(f"Vector size: {model.vector_size}")
print(f"Epochs: {model.epochs}")

Vocabulary size: 1737
Vector size: 100
Epochs: 5


In [37]:
### similar words

model.wv.similar_by_word('happy')

[('dear', 0.9990798830986023),
 ('day', 0.9990267157554626),
 ('great', 0.9987252950668335),
 ('wish', 0.9987207651138306),
 ('many', 0.9986615180969238),
 ('good', 0.9986341595649719),
 ('sweet', 0.9985784292221069),
 ('hope', 0.9985237121582031),
 ('of', 0.9985069632530212),
 ('morning', 0.9984884858131409)]

In [38]:
### define avgword2vec

def avgword2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key])

In [60]:
### apply avgword3vec on all sentences
from tqdm import tqdm

x = []
for i in tqdm(range(len(words))):
    #print("Hello",i)
    x.append(avgword2vec(words[i]))

print(x)

  4%|▍         | 244/5572 [00:00<00:04, 1258.61it/s]d:\KNMaterial\Complete-Data-Science\venv\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\KNMaterial\Complete-Data-Science\venv\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5572/5572 [00:02<00:00, 1901.09it/s]

[np.float32(0.006946371), np.float32(0.0049451906), np.float32(0.0067300578), np.float32(0.009215772), np.float32(0.00835505), np.float32(0.009546108), np.float32(0.009966818), np.float32(0.008183315), np.float32(0.008347938), np.float32(0.010100113), np.float32(0.008977836), np.float32(0.007819814), np.float32(0.009123387), np.float32(0.010344184), np.float32(0.00945042), np.float32(0.007817394), np.float32(0.008828514), np.float32(0.0070031104), np.float32(0.011326009), np.float32(0.0063395975), np.float32(0.01088345), np.float32(0.009748164), np.float32(0.0076923086), np.float32(0.0065372097), np.float32(0.010691903), np.float32(0.0067864265), np.float32(0.010626431), np.float32(0.010654561), np.float32(0.010802409), np.float32(0.00959114), np.float32(0.010829948), np.float32(0.011036059), np.float32(0.010942998), np.float32(0.012246074), np.float32(0.009716302), np.float32(0.008774126), np.float32(0.00914776), np.float32(0.009761885), np.float32(0.0055758795), np.float32(0.00969492

In [59]:
len(x)

5572

In [41]:
## convert to array
x_new = np.array(x)
x_new.shape

(5572,)

In [42]:
### dependent feature

messages['y'] = messages['label'].replace({'ham':1,'spam':0})
y = messages['y']

C:\Users\Shrinath\AppData\Local\Temp\ipykernel_15072\140591671.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  messages['y'] = messages['label'].replace({'ham':1,'spam':0})


In [43]:
len(y)

5572

In [48]:
## independent feature
## adding vectors of avgword2vec output to dataframe

df = pd.DataFrame(x)
df.head()

,0
0,0.006946
1,0.004945
2,0.006730
3,0.009216
4,0.008355


In [61]:
## Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")


Train set size: (4457, 1)
Test set size: (1115, 1)


In [62]:
## Train a classifier (Random Forest)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("\nTraining Random Forest classifier...")
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


Training Random Forest classifier...


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [63]:
## Make predictions
y_pred = clf.predict(X_test)

In [64]:
## Evaluate
print("\n" + "="*50)
print("MODEL EVALUATION")
print("="*50)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"\nConfusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred, target_names=['Spam', 'Ham'])}")



MODEL EVALUATION

Accuracy: 0.8287

Confusion Matrix:
[[ 60  89]
 [102 864]]

Classification Report:
              precision    recall  f1-score   support

        Spam       0.37      0.40      0.39       149
         Ham       0.91      0.89      0.90       966

    accuracy                           0.83      1115
   macro avg       0.64      0.65      0.64      1115
weighted avg       0.83      0.83      0.83      1115

